In [ ]:
from phi.jax import flow
import matplotlib.pyplot as plt
from tqdm import tqdm

N_TIME_STEPS = 150

def main():
    velocity = flow.StaggeredGrid(
        values=(0.0, 0.0),
        extrapolation=0.0,
        x=64,
        y=64,
        bounds=flow.Box(x=100, y=100),
    )
    smoke = flow.CenteredGrid(
        values=0.0,
        extrapolation=flow.extrapolation.BOUNDARY,
        x=200,
        y=200,
        bounds=flow.Box(x=100, y=100),
    )
    inflow = 0.2 * flow.CenteredGrid(
        values=flow.SoftGeometryMask(
            flow.Sphere(
                x=40,
                y=9.5,
                radius=5,
            )
        ),
        extrapolation=0.0,
        bounds=smoke.bounds,
        resolution=smoke.resolution,
    )

    @flow.math.jit_compile()
    def step(velocity_prev, smoke_prev, dt=1.0):
        smoke_next = flow.advect.mac_cormack(smoke_prev, velocity_prev, dt) + inflow
        buoyancy_force = smoke_next * (0.0, 0.1) @ velocity
        velocity_tent = flow.advect.semi_lagrangian(velocity_prev, velocity_prev, dt) + buoyancy_force * dt
        velocity_next, pressure = flow.fluid.make_incompressible(velocity_tent)
        return velocity_next, smoke_next

    plt.style.use("dark_background")

    for _ in tqdm(range(N_TIME_STEPS)):
        velocity, smoke = step(velocity, smoke)
        smoke_values_extracted = smoke.values.numpy("y,x")
        plt.imshow(smoke_values_extracted, origin="lower")
        plt.draw()
        plt.pause(0.01)
        plt.clf()


if __name__ == "__main__":
    main()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
start = datetime.datetime.now()

# Set random seed for reproducibility
np.random.seed(0)

# Number of particles
num_particles = 1000

# Initialize particle positions randomly within a 2D space of shape (1.0, 1.0)
positions = np.random.rand(num_particles, 2)

# Simulation parameters
num_steps = 100
dt = 0.01  # Time step
repulsion_constant = 0.01  # Adjust this parameter for stronger/weaker repulsion

# Simulation loop
for step in range(num_steps):
    # Calculate pairwise differences in positions
    delta_positions = positions[:, np.newaxis, :] - positions[np.newaxis, :, :]

    # Calculate distances between particles + small constant to avoid division by zero
    distances = np.sqrt((delta_positions**2).sum(axis=-1)) + 1e-8

    # Calculate normalized repulsion vectors
    repulsion_vectors = delta_positions / distances[..., np.newaxis]

    # Calculate forces using some repulsion law (in this case, inverse square law)
    forces = repulsion_constant * (1.0 / distances**2)[..., np.newaxis] * repulsion_vectors

    # Sum forces from all particles
    net_forces = forces.sum(axis=1)

    # Update positions based on forces (simple Euler integration)
    positions += dt * net_forces

    # Plot current state
#     plt.scatter(positions[:, 0], positions[:, 1])
#     plt.xlim(0, 1)
#     plt.ylim(0, 1)
#     plt.title(f'Step: {step + 1}')
#     plt.pause(0.01)
#     plt.clf()

# plt.show()
end = datetime.datetime.now()
print(end - start)
